<a id='top'></a>

# Watershed metrics for all Huc12 subwatersheds that intersect AWC recorded streams
Iterate over AKSSF regions and identify all HUC12 sub-watersheds that intersect an AWC recorded stream. Identify the downstream-most/outlet catchment for each Huc12 from this pool and convert the polygon to INSIDE centroid point.  Calculate the distance to coastline as the straight line distance in Km from centroid point to NHD recorded coastline and export this as a feature class/table.  Next use the outlet catchments unique identifier to query the appropriate dataset and build watersheds for each outlet catchment.  Calculate watershed metrics listed in the covariate section and export final merged csv using the catchment unique identifier field "cat_ID_con" to link the metric back to the source catchment/HUC12.  Merge watersheds together and use to calculate covariates.
## Covariates
Covariates needed for prediction on AWC-HUC12 outlets are as follows:
### Summer Precipitation
To be calculated in R using the outlet catchment centroid point feature class exported during outlet identification process.
### Watershed Slope Metrics
Regional Slope grids created in AKSSF_merge_grids.ipynb script.
Run zonal statistics on slope grid using merged watershed as zone feature.
Field names and descriptions:
* **awc_huc12s_wtd_slope_mn = mean watershed slope**
* **awc_huc12s_wtd_slope_min = min watershed slope**
* **awc_huc12s_wtd_slope_max = max watershed slope**
* **awc_huc12s_wtd_slope_sd (or cv) = standard deviation of watershed slope**
### Watershed Percent North Aspect
Regional North grids created in AKSSF_merge_grids.ipynb scripts.
North = aspects from 315-45 degrees and calculate the percentage of land area facing north for each watershed. Run tabulate area on north grid using merged watershed as zone feature and calculate percentage from area.
Field names and descriptions:
* **awc_huc12s_north_wtd = percent watershed with north aspect**
### Watershed Percent Lake Cover
Lakes feature classes for each network datatype (NHDPlus vs TauDEM) stored in AKSSF hydrography database on the T:
Calculate percentage of watershed that is covered by lakes/ponds using tabulate interesection between lake features and watersheds.
Field names and descriptions:
* **awc_huc12s_wtd_lake_per = percent watershed covered by lakes**
### Watershed Percent Glacier Cover
Use input glacier fc (from previous covariate calculations) stored in regional gdbs an calculate percent of watershed with glacial coverage using tabulate intersection between lake features and watersheds.
Field names and descriptions:
* **awc_huc12s_wtd_glac_per = percent watershed covered by glaciers**
### Watershed LCLD
LCLD rasters created in AKSSF_MODIS_lcld_ipynb.
Iterate over LCLD input rasters to produce yearly means for watersheds using zonal statistics.
Field names and descriptions:
* **awc_huc12s_wtd_lcld_mn_YYYY = mean lcld**


# FROM ORIGINAL CODE
## Aspect and Percent North Covariates
Calculate aspect grid with the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid, or we can't calculate covariates at
 the watershed scale.
 * **~~aspect_rch = calculate mean aspect for stream reach (zonal statistics)~~**
 * **~~aspect_cat = calculate mean aspect over catchments (zonal statistics)~~**


## Elevation Metrics
Calculate elevation metrics for catchment/watershed with temperature data using zonal statistics as table.
* <b> Running "ALL" zonal statistics as it does not allow you to choose two types of statistics
(ex statistics_type = ["MIN","MAX"]) instead set statistics_type="ALL"</b>
### Catchment Elevation Metrics
* **cat_elev_mn = mean elevation for catchment**
* **cat_elev_min = minimum elevation for catchment**
* **cat_elev_max = max elevation for catchment**
* **cat_elev_std = standard deviation of elevation for catchment**
### Watershed Elevation Metrics
* **wtd_elev_mn = mean watershed elevation**
* **wtd_elev_min = min watershed elevation**
* **wtd_elev_max = max watershed elevation**
* **wtd_elev_sd (or cv) = standard deviation of watershed elevation**
## Slope Metrics
Calculate slope metrics for catchment/watershed with temperature data using zonal statistics as table and
statistics_type = "ALL".
### Catchment Slope Metrics
* **cat_slope_mn = mean slope for catchment**
* **cat_slope_min = minimum slope for catchment**
* **cat_slope_max = max slope for catchment**
* **cat_slope_std = standard deviation of slope for catchment**

## Lake, Wetland and Glacier Cover
Lake/Pond type waterbodies from NHDPLus hydrography for those regions with NHDPlus derived datasets and NHD waterbodies
for those regions with TauDEM derived waterbodies.  Merge all waterbodies together (FTYPE = 390) and use tabulate area
calculate percent cover of lakes in catchments/watersheds.
 * **Created local copies of NHDPlus and NHD waterbodies and exported to T driver here <>  **
Use NLCD grid (wetlands.tif) and tabulate area on wetlands grid with watersheds (sum) / divide by number of cells in
watershed from fac grid, wetlands from NLCD



## Import modules
Set initial environments and import modules
Print system paths

In [1]:
import os, arcpy, sys,datetime
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())


imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\landcover', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\AppDa

## Functions
Define any functions that will be used

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput



## Section 1
### Set input datasets, output locations, and scratch workspaces
User to input paths for necessary input data and output locations
Scratch workspaces and output workspaces will be automatically created if they do not already exist.

In [3]:
# Get user inputs
# Used to format user inputs
inputDict = {"'":"",'"':""}

while True:
    try:
        userinput = replace_all((input('Input AKSSF parent directory containing regional sub-folders.\nLeave blank and hit enter to use the default location.\nDefault = D:\\GIS\\AKSSF\\') or 'D:\\GIS\\AKSSF'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            data_dir = userinput
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())
print(f'AKSSF parent directory set to {data_dir}\n {"-"*100}')
# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = J:\\GIS_data\\biota\\Aquatic\\Fauna\\AWC\\2021_Species_LifeStage.gdb\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "J:\\GIS_data\\biota\\Aquatic\\Fauna\\AWC\\2021_Species_LifeStage.gdb\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs\\"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())
print(f'AWC events feature class set to {awc_events}\n {"-"*100}')

# Enter output destination  - to create working folders and gdbs
while True:
    try:
        userinput3 = replace_all((input('Input path to create working folders.\nLeave blank and hit enter to use the default location.\nDefault = W:\\GIS\\') or 'W:\\GIS\\'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            temp_path = userinput3
            print(f'Output locations will be created at {temp_path}\n {"-"*100}')
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())

## Create working output location to store intermediate data
dirname = 'AKSSF_awcHuc12_cv'
tempgdbname = 'AKSSF_awcHuc12_cv.gdb'
temp_dir = os.path.join(temp_path, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print(f'Working Folder already created {temp_dir}\n {"-"*100}')

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print (f'Output location already exists{outcheck}\n {"-"*100}')
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print(f'Creating output GDB\n {"-"*100}')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print (f'Output geodatabase created at {outcheck}\n {"-"*100}')
    outgdb = tempgdb.getOutput(0)


AKSSF parent directory set to D:\GIS\AKSSF
 ----------------------------------------------------------------------------------------------------
AWC events feature class set to D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs
 ----------------------------------------------------------------------------------------------------
Output locations will be created at U:\GIS\
 ----------------------------------------------------------------------------------------------------
Creating output GDB
 ----------------------------------------------------------------------------------------------------
Output geodatabase created at U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb
 ----------------------------------------------------------------------------------------------------


## Section 1.1
### Set and create local copies of additional input data


In [4]:
import arcpy
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

nhdplusfol = []
tahuc12=[]

# Create and set HUC12 data if it does not already exist
nhdplushucs = os.path.join(outgdb, 'NHDPlusHUC12')
tauhucs = os.path.join(outgdb, 'NHD_H_HUC12')

if not arcpy.Exists(nhdplushucs):
    print(f'Huc12 data for NHDPlus Regions not yet created')
    #Enter NHDplus data folder
    while True:
        try:
            nhdplusfol = replace_all((input('Input path to source NHDPlus parent folder.\nLeave blank and hit enter to use the default location.\nDefault = J:\\GIS_data\\inlandWaters\\Hydrography\\NHD\\NHDPlus') or 'J:\\GIS_data\\inlandWaters\\Hydrography\\NHD\\NHDPlus'),inputDict)
            if not arcpy.Exists(nhdplusfol):
                print('Path specified does not exist!\nPlease re-enter a valid path')
                continue
            else:
                print(f'NHD HUC12 will be copied to {nhdplushucs}\n {"-"*100}')
                hucs = []
                walk = arcpy.da.Walk(nhdplusfol, datatype="FeatureClass", type="Polygon")

                for dirpath, dirnames, filenames in walk:
                    for filename in filenames:
                        if filename == 'WBDHU12':
                            hucs.append(os.path.join(dirpath, filename))
                arcpy.Merge_management(hucs,nhdplushucs,'','ADD_SOURCE_INFO')
                break
        except arcpy.ExecuteError:
            print(arcpy.GetMessages())
else:
    print(f'NHDPlus Hucs {nhdplushucs} located and exists = {arcpy.Exists(nhdplushucs)}')


if not arcpy.Exists(tauhucs):
    print(f'Huc12 data for Tau Regions not yet created')
    #Enter path to WBDHU12 from NHD_H gdb
    while True:
        try:
            tauhuc12 = replace_all((input('Input path to source WBDHU12 for state of Alaska.\nLeave blank and hit enter to use the default location.\nDefault = J:\\GIS_data\\inlandWaters\\Hydrography\\NHD\\NHD_H_Alaska_State_GDB.gdb\\WBD\\WBDHU12') or 'J:\\GIS_data\\inlandWaters\\Hydrography\\NHD\\NHD_H_Alaska_State_GDB.gdb\\WBD\\WBDHU12'),inputDict)
            if not arcpy.Exists(tauhuc12):
                print('Path specified does not exist!\nPlease re-enter a valid path')
                continue
            else:
                arcpy.CopyFeatures_management(tauhuc12,tauhucs)
                print(f'WBD Huc12  copied to {tauhucs}\n {"-"*100}')
                break
        except arcpy.ExecuteError:
                print(arcpy.GetMessages())

else:
    print(f'Tau Region Hucs {tauhucs} located and exists = {arcpy.Exists(tauhucs)}')


Huc12 data for NHDPlus Regions not yet created
NHD HUC12 will be copied to U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\NHDPlusHUC12
 ----------------------------------------------------------------------------------------------------
Huc12 data for Tau Regions not yet created
WBD Huc12  copied to U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\NHD_H_HUC12
 ----------------------------------------------------------------------------------------------------


## Section 2
### By Region
Identify downstream-most catchment for each Huc 12
 * Select by location and select catchment with most us contributing area
    * NHDPlus
        * Use update cursor to join TotalDrainageAreaSqKm from vaa table to catchment
        * Find max value from selection and save as outlet catchment for that HUC12
    * TauDEM
        * DSContArea - Drainage area at the downstream end of the link. Generally this is one grid cell upstream of the downstream end because the drainage area at the downstream end grid cell includes the area of the stream being joined.
 * Generate Centroid point and append to centroid dataset
    * Retain cat_id and Huc12-id
 * Append to HUC12 catchment dataset

In [5]:
import arcpy, time, os, datetime, operator

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces()

# Dictionaries and lists
nhdplusoutlets = []
nhdplusawccatouts = []
vaaDict = {}
catsDict = {}
huc12Dict = {}
nhdidDict = {}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
# rois = nhdplus_dat + tauDem_dat

# Or comment above and specify below specific subset
regions = ['D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River' ]

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    roi = os.path.basename(region)
    print(roi)
    if roi in nhdplus_dat:
        # Start roi time
        roi_start = time.time()
        hucs = nhdplushucs
        catsList = []
        outletList = []
        print(f'{roi} using data from {region} folder')
        # Set workspace to region folder
        arcpy.env.workspace = region
        gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
        sourcegdb = gdb[0]
        walk = arcpy.da.Walk(sourcegdb, datatype = ['FeatureClass','Table'])
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                if filename == 'cats_merge':
                    cats  = os.path.join(dirpath, filename)
                    append_value(catsDict,roi,cats)
                elif filename == 'vaa_merge':
                    vaas = os.path.join(dirpath, filename)
                    append_value(vaaDict, roi, vaas)
        #Output names and paths
        outletcatsname = roi + '_awc_huc12_catchment_outlets'
        outcatspath = os.path.join(outgdb,outletcatsname)
        outcatspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets')
        outletcatptsname = roi + '_awc_huc12_catchment_outlets_pts'
        outcatptspath = os.path.join(outgdb,outletcatptsname)
        outcatptspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets_pts')

        # Build Value dictionary to relate NHDPlus id to contributing area
        fields = ['NHDPlusID','TotDASqKm']
        fields2 = fields + ['cat_ID_con']
        valueDict = {int(r[0]):(r[1]) for r in arcpy.da.SearchCursor(vaas, fields)}
        where_clause=f'"MERGE_SRC" LIKE \'%{roi}%\''
        print(f'where_clause = {where_clause}')
        huclayer = arcpy.MakeFeatureLayer_management(hucs,'huclayer',where_clause = where_clause)
        print(f'{arcpy.GetCount_management(huclayer)} huc12s in {roi}')
        print(('*'*100))
        hucselect = arcpy.SelectLayerByLocation_management(huclayer,'INTERSECT',awc_events,'','SUBSET_SELECTION')
        print(('*'*100))
        print(f'{arcpy.GetCount_management(hucselect)} Huc12s in {roi} intersect awc events input')
        print(('*'*100))
        hucFields = [f for f in arcpy.ListFields(hucselect)]
        vcount =1
        with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
            for row in cur:
                print(f'Processing HUC {row[0]}')
                inhuc = row[1]
                cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                # Select by location using awc and huc 12
                arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                print(f'{vcount}. Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'NHDPlusID')]
                intersect = list(set(catList).intersection(valueDict))
                catDict = {int(i):(valueDict[i]) for i in intersect}
                # Find Catchment with max drainage area
                outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                append_value(huc12Dict, row[0], [int(outcatch),roi,valueDict[int(outcatch)]])
                append_value(nhdidDict,int(outcatch),[roi,row[0], valueDict[int(outcatch)]])
                outletList.append(int(outcatch))
                vcount+=1
            del(row)
        del(cur)

        outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
        out_expression ='"NHDPlusID" IN ' + str(tuple(outletList))
        #print(out_expression)
        outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
        print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Region {roi} at {outcatspath}')
        print(('*'*100))

        # Copy outputs
        arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,outgdb,outletcatsname)
        arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')
        # Create Copies to akssf data_dir regional gdbs also
        arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,sourcegdb,'awc_huc12_catchment_outlets')
        arcpy.FeatureToPoint_management(outcatspath2, outcatptspath2, 'INSIDE')
        nhdplusoutlets.append(outcatptspath)
        nhdplusawccatouts.append(outcatspath)
        # Add total drainage km from value dict to feature classes and cat_ID_con from regDict
        upfcs = [outcatspath, outcatptspath,outcatptspath2,outcatptspath2]
        for upfc in upfcs:
            arcpy.AddField_management(upfc,fields[1],'TEXT')
            arcpy.AddField_management(upfc,fields2[2],'TEXT')
            with arcpy.da.UpdateCursor(upfc,fields2) as cur:
                for row in cur:
                    row[1] = valueDict[row[0]]
                    row[2] = roi + '_' + str(int(row[0]))
                    cur.updateRow(row)
                del(row)
            del(cur)

        # End roi time
        roi_stop = time.time()
        roi_time = int (roi_stop - roi_start)
        print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=roi_time)})')
        print(f'{"*"*60}')

    else:
        #Section of code for taudem to follow
        print(('-'*100))
        print(f'Region {roi} will not be processed')
        print(('-'*100),'\n')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()



Cook_Inlet
Cook_Inlet using data from D:\GIS\AKSSF\Cook_Inlet folder
where_clause = "MERGE_SRC" LIKE '%Cook_Inlet%'
1058 huc12s in Cook_Inlet
****************************************************************************************************
****************************************************************************************************
661 Huc12s in Cook_Inlet intersect awc events input
****************************************************************************************************
Processing HUC 190202020501
1. Finding outlet for HUC 190202020501 out of 95 catchments.
************************************************************
Processing HUC 190202020503
2. Finding outlet for HUC 190202020503 out of 354 catchments.
************************************************************
Processing HUC 190202020507
3. Finding outlet for HUC 190202020507 out of 218 catchments.
************************************************************
Processing HUC 190202020504
4. Finding outlet for H

## Section 2.1
### Merge all outlet points together and calculate distance to coastline
Calculate Distance to Coast from outlet catchment point to the nearest coastline as a straight line distance
 * Generate near table and export as seperate csv

In [6]:
import arcpy, datetime
import numpy as pd

# Input path to coastline
coast = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\NHD_H_Alaska_Coastline_alb"

# Merge all catchment outlet centroids together
outletsname = 'AKSSF_NHDPlus_awcHuc12_outlet_cats_points'
outletspath = os.path.join(outgdb, outletsname)
all_nhd_outlet_pts = arcpy.Merge_management(nhdplusoutlets,outletspath)

if arcpy.Exists(coast):
    # Start timing function
    start = datetime.datetime.now()
    print(f'Getting distance to coast {datetime.datetime.now()}...')
    arcpy.analysis.Near(all_nhd_outlet_pts, coast, None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_DIST")
    arcpy.AlterField_management(all_nhd_outlet_pts,'NEAR_DIST','dist_catch_coast_km','dist_catch_coast_km' )
    arcpy.AddField_management(all_nhd_outlet_pts,'HUC12','TEXT')

    # Convert distance in meters to km
    with arcpy.da.UpdateCursor(all_nhd_outlet_pts,['dist_catch_coast_km','NHDPlusID','HUC12']) as cur:
        for row in cur:
            row[0] = row[0] * 0.001
            row[2] = nhdidDict[row[1]][1]
            cur.updateRow(row)
        del(row)
    del(cur)
else:
    print('Check path to coastal feature class')

stop = datetime.datetime.now()
elapsed  = stop - start
print (f'Process complete at {datetime.datetime.now()} time elapsed: {elapsed} ')

Getting distance to coast 2022-02-03 23:58:16.452432...
Process complete at 2022-02-03 23:58:49.310163 time elapsed: 0:00:32.857731 


### Convert to df and examine

In [7]:
import numpy
import pandas as pd

# Make catchment points df
cat_df = pd.DataFrame()
cat_field_list = []

for field in arcpy.ListFields(all_nhd_outlet_pts):
    cat_field_list.append(field.name)
cat_arr = arcpy.da.TableToNumPyArray(all_nhd_outlet_pts, ['cat_ID_con', 'NHDPlusID','dist_catch_coast_km','TotDASqKm','HUC12'])
cat_df = pd.DataFrame(cat_arr)
cat_df = cat_df.set_index('cat_ID_con')
cat_df

,NHDPlusID,dist_catch_coast_km,TotDASqKm,HUC12
cat_ID_con,,,,
Cook_Inlet_75004200000901,7.500420e+13,21.633384,36.96257508,190202020501
Cook_Inlet_75004200001724,7.500420e+13,17.599013,170.81217492,190202020503
Cook_Inlet_75004200001726,7.500420e+13,0.128861,570.1149751,190202020508
Cook_Inlet_75004200001493,7.500420e+13,0.147499,9.6406,190202020303
Cook_Inlet_75004200004105,7.500420e+13,0.137137,14.02302496,190202020102
...,...,...,...,...
Copper_River_75003900029086,7.500390e+13,0.389318,19.47842491,190201042003
Copper_River_75003900044552,7.500390e+13,0.355273,12.15134995,190201041803
Copper_River_75003900054944,7.500390e+13,0.063387,8.75949999,190201041807


### Export csv of outlet points for NHDPlus regions


In [8]:
import os
#Export CSV to read into R
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
nhd_catpts_outname = 'AKSSF_NHDPlus_awcHuc12_dist_catch_coast_km.csv'
arcpy.da.NumPyArrayToTable(cat_arr,os.path.join(outdir,nhd_catpts_outname))
print('CSV export complete')
print('----------')

CSV export complete
----------


## Section 3
Generate Watersheds

In [ ]:
# NHDPLUS Watersheds

import arcpy, time, datetime, os
import pandas as pd
from functools import reduce
import arcpy, time, os, datetime, operator

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces()
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

wtdList = []
wtdDict = {}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet','Copper_River']
#Limit to Cook inlet for testing
regions = ['D:\\GIS\\AKSSF\\Cook_Inlet']

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    reg_start = time.time()
    roi = os.path.basename(region)
    print(roi)
    if roi in nhdplus_dat:
        try:
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            ingdb = gdb[0]
            # set inputs
            vaa = os.path.join(ingdb, "vaa_merge")
            cats = os.path.join(ingdb, "cats_merge")
            streams = os.path.join(ingdb, "NHDFlowline_merge")
            outcats = os.path.join(ingdb, "awc_huc12_catchment_outlets")
            # Create list of nhdplus ids for outlet catchments
            idList = [int(row[0]) for row in arcpy.da.SearchCursor(outcats,'NHDPlusID')]
            #Make test list of few small catchments
            #idList = [75004400004166,75004400004344, 75004400010328]
            # Get list of index names for cats merge and add index if not already created
            index_names = [i.name for i in arcpy.ListIndexes(cats)]
            print(index_names)
            if 'NHDPlusID_index' not in index_names:
                print (f'Creating index for {cats}')
                arcpy.AddIndex_management(cats,'NHDPlusID','NHDPlusID_index')
            else:
                print(f'{cats} Indexed')

            #watersheds feature dataset for storing fcs
            fdatname = roi + '_Watersheds'
            fdat = os.path.join(outgdb,fdatname)

            if not arcpy.Exists(fdat):
                arcpy.management.CreateFeatureDataset(outgdb, fdatname, sr)
            else:
                print(f'{fdat} exists for {roi}')

            vaa_df1 = pd.DataFrame(arcpy.da.TableToNumPyArray(vaa, ("NHDPlusID", "FromNode", "ToNode", "StartFlag")))
            stream_df = pd.DataFrame(arcpy.da.TableToNumPyArray(streams, ("NHDPlusID", "FType")))
            dfs = [vaa_df1, stream_df]
            vaa_df = reduce(lambda left,right: pd.merge(left,right,on='NHDPlusID',how="outer"), dfs)
            # remove pipelines
            vaa_df = vaa_df[(vaa_df['FType'] != 428 )]
            vaa_df

            c=1
            for id in idList:
                iteration_start = time.time()
                print(f'{c}. Starting watershed for HUC {str(id)} ({(len(idList) - c)} remaining)')
                rec = [id]
                up_ids = []
                up_ids.append(rec)
                rec_len = len(rec)
                hws_sum = 0

                while rec_len != hws_sum:
                    fromnode = vaa_df.loc[vaa_df["NHDPlusID"].isin(rec), "FromNode"]
                    rec = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "NHDPlusID"]
                    rec_len = len(rec)
                    rec_hws = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "StartFlag"]
                    hws_sum = sum(rec_hws)
                    up_ids.append(rec)
                #up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
                newup_ids = []
                for x in up_ids:
                    newup_ids.extend(x)

                tempLayer = "catsLyr"
                expression = '"NHDPlusID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
                arcpy.MakeFeatureLayer_management(cats, tempLayer, where_clause=expression)
                outdis = "memory/wtd_" + str(round(id))
                outwtd = os.path.join(fdat,f'{roi}_wtd_{str(int(id))}')

                dis = arcpy.Dissolve_management(tempLayer, outdis)
                watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
                wtdList.append(outwtd)
                append_value(wtdDict,roi,outwtd)


                # Stop iteration timer
                iteration_stop = time.time()
                iter_time = int (iteration_stop - iteration_start)
                print(f'Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
                print(f'{"*"*60}')

                c=c+1
            wtd_merge = arcpy.Merge_management(wtdList, os.path.join(ingdb,'awchuc12_wtds_merge'),'','ADD_SOURCE_INFO')
            arcpy.AddField_management(wtd_merge,'cat_ID_con','TEXT')
            arcpy.AddField_management(wtd_merge,'cat_ID','DOUBLE')
            arcpy.AddField_management(wtd_merge,'cat_ID_txt','TEXT')
            arcpy.AddField_management(wtd_merge,'NHDPlusID','DOUBLE')
            with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','NHDPlusID','cat_ID_con','cat_ID','cat_ID_txt']) as cur:
                for row in cur:
                    # Pull nhdplus id from merge source and calculate fields
                    nhdplusid= int(row[0].split('_')[-1])
                    row[1] = nhdplusid
                    row[2] = roi + '_' + str(nhdplusid)
                    row[3] = nhdplusid
                    row[4] = str(nhdplusid)
                    cur.updateRow(row)
                del(row)
            del(cur)
            arcpy.CopyFeatures_management(wtd_merge,os.path.join(outgdb,f'{roi}_awchuc12_wtds_merge' ))

            # Stop iteration timer
            reg_stop = time.time()
            reg_time = int (reg_stop - reg_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=reg_time)})')
            print(f'{"*"*100}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')




Cook_Inlet
['FDO_OBJECTID', 'FDO_Shape', 'NHDPlusID_index']
D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\cats_merge Indexed
U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\Cook_Inlet_Watersheds exists for Cook_Inlet
1. Starting watershed for HUC 75004200000901 (660 remaining)
Elapsed time: (0:00:03)
************************************************************
2. Starting watershed for HUC 75004200001724 (659 remaining)
Elapsed time: (0:00:04)
************************************************************
3. Starting watershed for HUC 75004200001726 (658 remaining)
Elapsed time: (0:00:08)
************************************************************
4. Starting watershed for HUC 75004200001493 (657 remaining)
Elapsed time: (0:00:03)
************************************************************
5. Starting watershed for HUC 75004200004105 (656 remaining)
Elapsed time: (0:00:03)
************************************************************
6. Starting watershed for HUC 75004200009084 (655 remaining)


In [ ]:
from arcpy.sa import *

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338) #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_wet_taba_tables = []
cat_elev_ztables = []
wtd_elev_ztables = []
cat_slope_ztables = []
wtd_slope_ztables = []


# Clear lists
cat_cur_fields = []
wtd_cur_fields = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
#Limit to ci for testing
regions = ['D:\\GIS\\AKSSF\\Cook_Inlet']

for region in regions:
    roi = os.path.basename(region)
    if roi in str(region):
        print(f'{roi} using data from {region} folder')
        # Set data and variables unique to regions with NHDPlus Data
        if roi in nhdplus_dat:
            lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb"
            # Fields for update cursor
            cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
            wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
            print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
            print(f'{"*"*100}')
        # Set data and variables unique to regions with TauDEM Data
        elif roi in tauDem_dat:
            lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb"
            # Fields for update cursor
            if roi == 'Bristol_Bay':
                cat_cur_fields = ['cat_ID_txt', 'catID',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
            else:
                cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
            print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
            print(f'{"*"*100}')
        # Start iter timing function
        iteration_start = time.time()
        # Set workspace to region folder
        arcpy.env.workspace = region
        walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                # Set merged watersheds dataset
                if 'awchuc12_wtds_merge'== filename:
                    wtdpath = os.path.join(dirpath,filename)
                    wtdname = roi +'_'+ filename
                    # Make local copy projected in AKAlbers
                    wtd_merge = arcpy.FeatureClassToFeatureClass_conversion(wtdpath,outgdb,wtdname)
                    print(f'Merged watershed dataset {filename} found')
                    print(f'{"*"*100}')
                    wtdfieldnames = []
                    wtdlstFields = arcpy.ListFields(wtd_merge)
                    for field in wtdlstFields:
                        wtdfieldnames.append(field.name)
                    if str(wtd_cur_fields[0]) in wtdfieldnames:
                        print (f'{wtd_cur_fields[0]} field already in dataset')
                        print(f'{"*"*100}')
                    else:
                        print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                        print(f'{"*"*100}')
                        # add cat_ID_txt field and concat cat_ID + region
                        arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                        # populate cat_ID_txt
                        with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                            for row in cur:
                                strval = str(row[1])
                                row[0] = strval.replace('.0',"")
                                # Update rows
                                cur.updateRow(row)
                            del(row)
                        del(cur)
                    if str(wtd_cur_fields[2]) in wtdfieldnames:
                        print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                        print(f'{"*"*100}')
                    else:
                        print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                        print(f'{"*"*100}')
                        # add cat_ID_con field and concat cat_ID + region
                        arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                        # populate cat_ID_txt
                        with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                            for row in cur:
                                strval = str(row[1])
                                row[2] = str(roi) +'_'+ strval.replace(".0","")
                                # Update rows
                                cur.updateRow(row)
                            del(row)
                        del(cur)

                # Select glaciers fc
                elif 'glaciers' == filename:
                    # Make local copy projected in AKAlbers
                    glacpath = os.path.join(dirpath, filename)
                    glacname = roi+'_'+filename
                    glac_fc = arcpy.FeatureClassToFeatureClass_conversion(glacpath,outgdb,glacname)

                # Select elevation raster
                elif 'elev.tif' == filename:
                    elev_rast = os.path.join(dirpath, filename)

                # # Select aspect raster
                # elif 'aspect' in filename:
                #     asp_rast = os.path.join(dirpath, filename)

                # Select north raster
                elif 'north.tif' == filename:
                    nor_rast = os.path.join(dirpath, filename)

                # Select slope raster
                elif 'slope.tif' == filename:
                    slope_rast = os.path.join(dirpath, filename)

                # Select wetland raster
                elif 'wetlands.tif' == filename:
                    wet_rast = os.path.join(dirpath, filename)

                # Select catch_int fc (catchments of interest for region) and make a copy
                elif 'awc_huc12_catchment_outlets' == filename:
                    # Make local copy projected in AKAlbers
                    catspath = os.path.join(dirpath,filename)
                    catsname = roi +"_"+filename
                    cats = arcpy.FeatureClassToFeatureClass_conversion(catspath, outgdb,catsname)
                    catlstfields = arcpy.ListFields(cats)
                    catfieldnames = []
                    for field in catlstfields:
                        catfieldnames.append(field.name)
                    if str(cat_cur_fields[0]) in catfieldnames:
                        print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                        print(f'{"*"*100}')
                    else:
                        print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                        print(f'{"*"*100}')
                        # add cat_ID_txt field
                        arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                        # populate cat_ID_txt
                        with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                            for row in cur:
                                strval = str(row[1])
                                row[0] = strval.replace('.0',"")
                                # Update rows
                                cur.updateRow(row)
                            del(row)
                        del(cur)
                    if str(cat_cur_fields[2]) in catfieldnames:
                        print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                        print(f'{"*"*100}')
                    else:
                        print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                        print(f'{"*"*100}')
                        # add cat_ID_txt field & cat_ID + region concat field
                        arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                        # populate cat_ID_con
                        with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                            for row in cur:
                                strval = str(row[1])
                                row[2] = str(roi) +'_'+ strval.replace('.0',"")
                                # Update rows
                                cur.updateRow(row)
                            del(row)
                        del(cur)

        print (f'Calculating topographic metrics for catchments & watersheds of interest in {roi} region')
        print ('----------')
        print(f'Geodatabase: {outgdb}')
        print ('----------')
        print (f'Watershed Merge: {wtd_merge}')
        print (f'  Projection {arcpy.Describe(wtd_merge).spatialReference.name}')
        print ('----------')
        print (f'Catchment Intersect: {cats}')
        print (f'  Projection {arcpy.Describe(cats).spatialReference.name}')
        print ('----------')
        print (f'Elevation Raster: {elev_rast}')
        print (f'  Projection: {arcpy.Describe(elev_rast).spatialReference.name}')
        print ('----------')
        print (f'North Aspect Raster: {nor_rast}')
        print (f'  Projection: {arcpy.Describe(nor_rast).spatialReference.name}')
        print ('----------')
        print (f'Wetlands Raster: {wet_rast}')
        print (f'  Projection {arcpy.Describe(wet_rast).spatialReference.name}')
        print ('----------')
        print (f'Slope Raster: {slope_rast}')
        print (f'  Projection {arcpy.Describe(slope_rast).spatialReference.name}')
        print ('----------')
        print (f'Lakes Ponds fc: {lakes_fc}')
        print (f'  Projection {arcpy.Describe(lakes_fc).spatialReference.name}')
        print ('----------')
        print (f'Glaciers fc: {glac_fc} ')
        print (f'  Projection {arcpy.Describe(glac_fc).spatialReference.name}')
        print ('----------')
        print (f'{arcpy.GetCount_management(wtd_merge)} Watersheds to process')
        print ('----------')
        print (f'Catchment intersect {cats} selected')
        print ('----------')

        try:

            # # Aspect variables
            # wtd_merge_asp_table_name = roi + "awc_huc12_Watersheds_Merge_AspectZstats"
            # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
            # cat_asp_table_name = roi + "awc_huc12_Catchments_AspectZstats"
            # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

            # Percent North variables
            wtd_merge_pernorth_table_name = roi + "awc_huc12_Watersheds_Merge_PercentNorth"
            wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
            # cat_pernorth_table_name = roi + "awc_huc12_Catchments_PercentNorth"
            # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

            # Elevation variables
            wtd_merge_elev_table_name = roi + "awc_huc12_Watersheds_Merge_ElevZstats"
            wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
            cat_elev_table_name = roi + "awc_huc12_Catchments_ElevZstats"
            cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)

            # Slope variables
            wtd_merge_slope_table_name = roi + "awc_huc12_Watershed_Merge_SlopeZstats"
            wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
            cat_slope_table_name = roi + "awc_huc12_Catchments_SlopeZstats"
            cat_slope_table_path = os.path.join(outgdb, cat_slope_table_name)

            # Lakes Ponds variables
            wtd_merge_lp_table_name = roi + "awc_huc12_Watershed_Merge_LakesPonds"
            wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
            cat_lp_table_name = roi + "awc_huc12_Catchments_LakesPonds"
            cat_lp_path = os.path.join(outgdb, cat_lp_table_name)

            # Wetlands variables
            wtd_merge_wetlands_table_name = roi + "awc_huc12_Watershed_Merge_Wetlands"
            wtd_merge_wetlands_table_path = os.path.join(outgdb, wtd_merge_wetlands_table_name)
            cat_wetlands_table_name = roi + "awc_huc12_Catchments_Wetlands"
            cat_wetlands_table_path = os.path.join(outgdb, cat_wetlands_table_name)

            # Glaciers
            wtd_merge_glac_table_name = roi + "awc_huc12_Watershed_Merge_Glaciers"
            wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)
            cat_glac_table_name = roi + "awc_huc12_Catchments_Glaciers"
            cat_glac_table_path = os.path.join(outgdb, cat_glac_table_name)

            # Begin Zonal Stats
            zstat_start = time.time()
            print(f'Begin Elevation zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                  f' region')

            # Elevation Zonal statistics  for watersheds
            print(f'Calculating {roi} watershed elevation zonal stats...')
            arcpy.env.snapRaster = elev_rast
            arcpy.env.cellSize = elev_rast
            wtd_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                            zone_field = wtd_cur_fields[0],
                                                            in_value_raster = elev_rast,
                                                            out_table = wtd_merge_elev_table_path,
                                                            statistics_type='ALL'
                                                            )
            # Add region identifier field for watershed tables                                                )
            arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
            # Add cat_ID_Con field
            arcpy.AddField_management(wtd_elev_metrics_table,'cat_ID_con',field_type='TEXT')
            # Update region field
            with arcpy.da.UpdateCursor(wtd_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                for row in cur:
                    row[0] = roi
                    strval = str(row[1])
                    row[2] = roi+"_"+strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)
            # Append watershed elev table to list
            wtd_elev_ztables.append(wtd_elev_metrics_table)

            # Elevation zonal statistics for catchments
            print(f'Calculating {roi} catchment elevation zonal stats...')
            arcpy.env.snapRaster = elev_rast
            arcpy.env.cellSize = elev_rast
            cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                            zone_field = cat_cur_fields[0],
                                                            in_value_raster = elev_rast,
                                                            out_table = cat_elev_table_path,
                                                            statistics_type='ALL'
                                                            )
            # Add region identifier field for catchment table
            arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')
            # Add cat_ID_Con field
            arcpy.AddField_management(cat_elev_metrics_table,'cat_ID_con',field_type='TEXT')

            # Update fields
            with arcpy.da.UpdateCursor(cat_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                for row in cur:
                    row[0] = roi
                    strval = str(row[1])
                    row[2] = roi+"_"+strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)
            # Append catchment elev table to list
            cat_elev_ztables.append(cat_elev_metrics_table)

            # Slope zonal statistics for catchments
            print(f'Calculating {roi} catchment slope zonal stats...')
            arcpy.env.snapRaster = slope_rast
            arcpy.env.cellSize = slope_rast
            cat_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                            zone_field = cat_cur_fields[0],
                                                            in_value_raster = slope_rast,
                                                            out_table = cat_slope_table_path,
                                                            statistics_type='ALL'
                                                            )
            # Add region identifier field for catchment table
            arcpy.AddField_management(cat_slope_metrics_table,'region',field_type='TEXT')
            # Add cat_ID_Con field
            arcpy.AddField_management(cat_slope_metrics_table,'cat_ID_con',field_type='TEXT')

            # Update region field
            with arcpy.da.UpdateCursor(cat_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                for row in cur:
                    row[0] = roi
                    strval =str(row[1])
                    row[2] = roi+"_"+strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)
            # Append catchment slope table to list
            cat_slope_ztables.append(cat_slope_metrics_table)

            # Watershed slope Zonal Statistics
            print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                  f' region')

            # Elevation Zonal statistics  for watersheds
            print(f'Calculating {roi} watershed slope zonal stats...')
            arcpy.env.snapRaster = slope_rast
            arcpy.env.cellSize = slope_rast
            wtd_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                            zone_field = wtd_cur_fields[0],
                                                            in_value_raster = slope_rast,
                                                            out_table = wtd_merge_slope_table_path,
                                                            statistics_type='ALL'
                                                            )
            # Add region identifier field for watershed tables                                                )
            arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
            # Add cat_ID_Con field
            arcpy.AddField_management(wtd_slope_metrics_table,'cat_ID_con',field_type='TEXT')

            # Update region field
            with arcpy.da.UpdateCursor(wtd_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                for row in cur:
                    row[0] = roi
                    strval = str(row[1])
                    row[2] = roi+"_"+strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)
            # Append watershed slope table to list
            wtd_slope_ztables.append(wtd_slope_metrics_table)

            # # Aspect Zonal statistics  for watersheds
            # print(f'Calculating {roi} watershed aspect zonal stats...')
            # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
            #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
            #                                                statistics_type='ALL')
            # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
            # Add cat_ID_Con field
            # arcpy.AddField_management(wtd_asp_metrics_table,'cat_ID_con',field_type='TEXT')
            # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
            # Update region field
            # with arcpy.da.UpdateCursor(wtd_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
            #     for row in cur:
            #         row[0] = roi
            #         strval = str(row[1])
            #         row[2] = roi+"_"+strval.replace(".0","")
            #         # Update
            #         cur.updateRow(row)
            #     del(row)
            # del(cur)
            # wtd_asp_ztables.append(wtd_asp_metrics_table)

            # # Aspect Zonal statistics for catchments
            # print(f'Calculating {roi} catchment aspect zonal stats...')
            # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
            #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
            #                                                statistics_type='ALL')
            # arcpy.AddField_management(cat_asp_metrics_table, 'region', field_type='TEXT')
            # Add cat_ID_Con field
            # arcpy.AddField_management(cat_asp_metrics_table,'cat_ID_con',field_type='TEXT')
            # arcpy.CalculateField_management(cat_asp_metrics_table, 'region', 'roi')
            # Update region field
            # with arcpy.da.UpdateCursor(cat_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
            #     for row in cur:
            #         strval = str(row[1])
            #         row[2] = roi+"_"+strval.replace(".0","")
            #         # Update
            #         cur.updateRow(row)
            #     del(row)
            # del(cur)
            # cat_asp_ztables.append(cat_asp_metrics_table)

            zstat_stop = time.time()
            zstat_time = int (zstat_stop - zstat_start)
            print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
            print(f'{"*"*100}')

            # Tabulate Area with north grid and watersheds
            tabarea_start = time.time()
            print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
            print(f'{"*"*100}')
            # Percent North Tabulate area for watersheds
            wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                          zone_field= wtd_cur_fields[0],
                                                          in_class_data=nor_rast,
                                                          class_field="Value",
                                                          out_table = wtd_merge_pernorth_table_path
                                                          )
            # Add region and percent north fields
            arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
            arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
            arcpy.AddField_management(wtd_per_north_tabarea, 'awc_huc12s_wtd_north_per', field_type='Float')
            arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
            arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
            wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
            print (wtdnorfields)
            with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
                for row in cur:
                    strval = str(row[1])
                    row[4] = roi
                    row[5] = row[3]/(row[3]+row[2])*100
                    row[6] = strval.replace('.0','')
                    row[7] = roi +'_'+ strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)
            # Drop UPPERCASE field form tab area
            arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
            # Append watershed percent north table to list
            wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

            # Percent Lakes Ponds using Tabulate Intersection for watersheds
            print(f'Begin tabulate intersection between {lakes_fc} and watersheds in {roi} region')
            print(f'{"*"*100}')
            wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                zone_fields=wtd_cur_fields[0],
                                                                in_class_features=lakes_fc,
                                                                out_table=wtd_merge_lp_table_path,
                                                                class_fields='Ftype',
                                                                out_units="SQUARE_METERS"
                                                                )
            # Add region and cat id fields
            arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','awc_huc12s_wtd_lake_per','awc_huc12s_wtd_lake_per')
            arcpy.AlterField_management(wtd_lp_tabint,'AREA','awc_huc12s_wtd_lake_area_sqm','awc_huc12s_wtd_lake_area_sqm')
            arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
            arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
            arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
            wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
            print (wtdlpfields)
            with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
                for row in cur:
                    strval = str(row[1])
                    row[5] = roi
                    row[6] = strval.replace('.0','')
                    row[7] = roi +'_'+ strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)

            # Append watershed lakes ponds table to list
            wtd_lp_tabint_tables.append(wtd_lp_tabint)

            # Percent glaciers using Tabulate Intersection for watersheds
            print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
            print(f'{"*"*100}')
            wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                zone_fields=wtd_cur_fields[0],
                                                                in_class_features=glac_fc,
                                                                out_table=wtd_merge_glac_table_path,
                                                                class_fields='O1Region',
                                                                out_units="SQUARE_METERS"
                                                                )
            # Add region and cat id fields
            arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','awc_huc12s_wtd_glacier_per','awc_huc12s_wtd_glacier_per')
            arcpy.AlterField_management(wtd_glac_tabint,'AREA','awc_huc12s_wtd_glacier_area_sqm','awc_huc12s_wtd_glacier_area_sqm')
            arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
            arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
            arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
            wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
            print (wtdglacfields)
            with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
                for row in cur:
                    strval = str(row[1])
                    row[5] = roi
                    row[6] = strval.replace('.0','')
                    row[7] = roi +'_'+ strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)
            # Append watershed percent glacier table to list
            wtd_glac_tabint_tables.append(wtd_glac_tabint)

            # Tabulate Area with wetlands grid and watersheds
            print(f'Begin tabulate intersection between {wet_rast} and watersheds in {roi} region')
            print(f'{"*"*100}')
            # Wetlands tabulate area for watersheds
            wtd_per_wet_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                          zone_field= wtd_cur_fields[0],
                                                          in_class_data=wet_rast,
                                                          class_field="Value",
                                                          out_table=wtd_merge_wetlands_table_path
                                                          )
            # Add region and percent wet fields
            arcpy.AlterField_management(wtd_per_wet_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
            arcpy.AddField_management(wtd_per_wet_tabarea, 'region', field_type='TEXT')
            arcpy.AddField_management(wtd_per_wet_tabarea, 'awc_huc12s_wtd_wet_per', field_type='Float')
            arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[0], field_type='TEXT')
            arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[2], field_type='TEXT')
            wtdwetfields = [f.name for f in arcpy.ListFields(wtd_per_wet_tabarea)]
            print (wtdwetfields)
            with arcpy.da.UpdateCursor(wtd_per_wet_tabarea, wtdwetfields) as cur:
                for row in cur:
                    strval = str(row[1])
                    row[4] = roi
                    row[5] = row[3]/(row[3]+row[2])*100
                    row[6] = strval.replace('.0','')
                    row[7] = roi +'_'+ strval.replace(".0","")
                    # Update
                    cur.updateRow(row)
                del(row)
            del(cur)
            # Drop UPPERCASE field form tab area
            arcpy.DeleteField_management(wtd_per_wet_tabarea,'CAT_ID_TXT_DEL')
            # Append watershed percent wetlands table to list
            wtd_wet_taba_tables.append(wtd_per_wet_tabarea)

            # # Percent North Tabulate Area for catchments
            # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_con',
            #                                             in_class_data=nor_rast,"Value",
            #                                             out_table=cat_pernorth_table_path)

            # # Add and calculate region identifier field for catchment table
            # arcpy.AlterField_management(cat_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
            # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
            # arcpy.AddField_management(cat_per_north_tabarea, 'cat_north_per', field_type='Float')
            # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[0], field_type='TEXT')
            # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[2], field_type='TEXT')
            # catnorfields = [f.name for f in arcpy.ListFields(cat_per_north_tabarea)]
            # print (catnorfields)
            # with arcpy.da.UpdateCursor(cat_per_north_tabarea,catnorfields) as cur:
            #     for row in cur:
            #         strval = str(row[1])
            #         row[4] = roi
            #         row[5] = row[3]/(row[3]+row[2])*100
            #         row[6] = strval.replace('.0','')
            #         row[7] = roi +'_'+ strval.replace(".0","")
            #         # Update
            #         cur.updateRow(row)
            #     del(row)
            # del(cur)
            # Drop UPPERCASE field form tab area
            # arcpy.DeleteField_management(cat_per_north_tabarea,'CAT_ID_TXT_DEL')
            # # Append catchment percent north table to list
            # cat_pernorth_taba_tables.append(cat_per_north_tabarea)

            tabarea_stop = time.time()
            tabarea_time = int (tabarea_stop - tabarea_start)
            print(f'Tabulate area/intersections for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
            print(f'{"*"*100}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

        iter_stop = time.time()
        iter_time = int(iter_stop - iteration_start)
        print(f'All Covariates for {roi} completed. Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
        print(f'{"*"*100}')

else:
    print(f'Region {str(roi)} not found in {region}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')



## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [ ]:
# Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_awc_huc12_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_awc_huc12_cat_elev')
cat_slope_table_out = os.path.join(outgdb,'AKSSF_awc_huc12_cat_slope')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_awc_huc12_wtd_elev')
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_awc_huc12_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_awc_huc12_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_awc_huc12_wtd_slope')
wtd_wet_table_out = os.path.join(outgdb, 'AKSSF_awc_huc12_wtd_wetland_per')

# Merge all regional tables together
outtables = []
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
arcpy.AlterField_management(wtd_per_north,"VALUE_0","awc_huc12s_non_north_area","awc_huc12s_non_north_area")
arcpy.AlterField_management(wtd_per_north,"VALUE_1","awc_huc12s_north_area","awc_huc12s_north_area")
outtables.append(wtd_per_north)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
outtables.append(cat_elev)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)
outtables.append(wtd_elev)
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
outtables.append(wtd_slope)
cat_slope = arcpy.Merge_management(cat_slope_ztables, cat_slope_table_out)
outtables.append(cat_slope)
wtd_wet = arcpy.Merge_management(wtd_wet_taba_tables, wtd_wet_table_out)
arcpy.AlterField_management(wtd_wet,"VALUE_0","awc_huc12s_non_wetland_area","awc_huc12s_non_wetland_area")
arcpy.AlterField_management(wtd_wet,"VALUE_1","awc_huc12s_wetland_area","awc_huc12s_wetland_area")
outtables.append(wtd_wet)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
outtables.append(wtd_glac)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)
outtables.append(wtd_lp)
print ('Tables merged')
print('----------')


In [ ]:
#Set up field dictionary
elevDict = { 'ZONE_CODE': ('awc_huc12s_cat_elev_ZONE_CODE', 'awc_huc12s_wtd_elev_ZONE_CODE'),
         'COUNT': ('awc_huc12s_cat_elev_COUNT', 'awc_huc12s_wtd_elev_COUNT'),
          'AREA': ('awc_huc12s_cat_elev_AREA', 'awc_huc12s_wtd_elev_AREA'),
          'MIN': ('awc_huc12s_cat_elev_MIN', 'awc_huc12s_wtd_elev_MIN'),
          'MAX': ('awc_huc12s_cat_elev_MAX', 'awc_huc12s_wtd_elev_MAX'),
          'RANGE': ('awc_huc12s_cat_elev_RANGE', 'awc_huc12s_wtd_elev_RANGE'),
          'MEAN': ('awc_huc12s_cat_elev_MEAN', 'awc_huc12s_wtd_elev_MEAN'),
          'STD': ('awc_huc12s_cat_elev_STD', 'awc_huc12s_wtd_elev_STD'),
          'SUM': ('awc_huc12s_cat_elev_SUM', 'awc_huc12s_wtd_elev_SUM'),
          'VARIETY': ('awc_huc12s_cat_elev_VARIETY', 'awc_huc12s_wtd_elev_VARIETY'),
          'MAJORITY': ('awc_huc12s_cat_elev_MAJORITY', 'awc_huc12s_wtd_elev_MAJORITY'),
          'MINORITY': ('awc_huc12s_cat_elev_MINORITY', 'awc_huc12s_wtd_elev_MINORITY'),
          'MEDIAN': ('awc_huc12s_cat_elev_MEDIAN', 'awc_huc12s_wtd_elev_MEDIAN'),
          'PCT90': ('awc_huc12s_cat_elev_PCT90', 'awc_huc12s_wtd_elev_PCT90')
         }

slopeDict = { 'ZONE_CODE': ('awc_huc12s_cat_slope_ZONE_CODE', 'awc_huc12s_wtd_slope_ZONE_CODE'),
         'COUNT': ('awc_huc12s_cat_slope_COUNT', 'awc_huc12s_wtd_slope_COUNT'),
          'AREA': ('awc_huc12s_cat_slope_AREA', 'awc_huc12s_wtd_slope_AREA'),
          'MIN': ('awc_huc12s_cat_slope_MIN', 'awc_huc12s_wtd_slope_MIN'),
          'MAX': ('awc_huc12s_cat_slope_MAX', 'awc_huc12s_wtd_slope_MAX'),
          'RANGE': ('awc_huc12s_cat_slope_RANGE', 'awc_huc12s_wtd_slope_RANGE'),
          'MEAN': ('awc_huc12s_cat_slope_MEAN', 'awc_huc12s_wtd_slope_MEAN'),
          'STD': ('awc_huc12s_cat_slope_STD', 'awc_huc12s_wtd_slope_STD'),
          'SUM': ('awc_huc12s_cat_slope_SUM', 'awc_huc12s_wtd_slope_SUM'),
          'VARIETY': ('awc_huc12s_cat_slope_VARIETY', 'awc_huc12s_wtd_slope_VARIETY'),
          'MAJORITY': ('awc_huc12s_cat_slope_MAJORITY', 'awc_huc12s_wtd_slope_MAJORITY'),
          'MINORITY': ('awc_huc12s_cat_slope_MINORITY', 'awc_huc12s_wtd_slope_MINORITY'),
          'MEDIAN': ('awc_huc12s_cat_slope_MEDIAN', 'awc_huc12s_wtd_slope_MEDIAN'),
          'PCT90': ('awc_huc12s_cat_slope_PCT90', 'awc_huc12s_wtd_slope_PCT90')
         }

# Rename fields for elevation tables
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][1]
        newalias = elevDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][0]
        newalias = elevDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)

for field in arcpy.ListFields(cat_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][0]
        newalias = slopeDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_slope, keyval, newname, newalias)

In [ ]:
# # Export copies of dbf tables as csv
# outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
# for table in outtables:
#     tablename = arcpy.Describe(table).basename + ".csv"
#     tablepath = os.path.join(outdir,tablename)
#     print( tablepath)
#     arcpy.conversion.TableToTable(table, outdir, tablename)

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
# list to store covariate data frames
dfs = []

In [ ]:
# Make catchment elev df
cat_df = pd.DataFrame()
cat_field_list = []
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","awc_huc12s_cat_elev_ZONE_CODE"],axis=1)
cat_df = cat_df.set_index('cat_ID_con')
dfs.append(cat_df)
cat_df


In [ ]:
# Make catchment slope df
cat_sl_df = pd.DataFrame()
cat_sl_field_list = []
for field in arcpy.ListFields(cat_slope):
    cat_sl_field_list.append(field.name)
cat_sl_arr = arcpy.da.TableToNumPyArray(cat_slope, cat_sl_field_list)
cat_sl_df = pd.DataFrame(cat_sl_arr)
cat_sl_df = cat_sl_df.drop(["OBJECTID", "awc_huc12s_cat_slope_ZONE_CODE"],axis=1)
cat_sl_df = cat_sl_df.set_index('cat_ID_con')
dfs.append(cat_sl_df)
cat_sl_df

In [ ]:
# Make watershed elev df
wtd_df = pd.DataFrame()
wtd_field_list = []
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)
wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","awc_huc12s_wtd_elev_ZONE_CODE"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_con')
dfs.append(wtd_df)
wtd_df

In [ ]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
wtd_sl_arr = arcpy.da.TableToNumPyArray(wtd_slope, wtd_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID", "awc_huc12s_wtd_slope_ZONE_CODE"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_con')
dfs.append(wtd_sl_df)
wtd_sl_df

In [ ]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_con')
dfs.append(wtd_n_df)
wtd_n_df

In [ ]:
# Make watershed wetland df
wtd_wet_df = pd.DataFrame()
wtd_wet_field_list = []
for field in arcpy.ListFields(wtd_wet):
    wtd_wet_field_list.append(field.name)
wtd_wet_arr = arcpy.da.TableToNumPyArray(wtd_wet,wtd_wet_field_list)
wtd_wet_df = pd.DataFrame(wtd_wet_arr)
wtd_wet_df = wtd_wet_df.drop("OBJECTID",axis=1)
wtd_wet_df = wtd_wet_df.set_index('cat_ID_con')
dfs.append(wtd_wet_df)
wtd_wet_df

In [ ]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop("OBJECTID",axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_con')
dfs.append(wtd_lp_df)
wtd_lp_df

In [ ]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop("OBJECTID",axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_con')
dfs.append(wtd_glac_df)
wtd_glac_df

## Merge all covariate dataframes together and drop unnecessary columns
 * Recalculate cat_ID as float64 type
 * Reorder columns
 * Export final csv


In [ ]:
# Merge all data frames together
import numpy as np
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)
df_final.columns = list(uniquify(df_final))
#List of final columns in the order to output
final_cols = ['cat_ID_txt','cat_ID','region', 'awc_huc12s_cat_slope_COUNT', 'awc_huc12s_cat_slope_AREA', 'awc_huc12s_cat_slope_MIN', 'awc_huc12s_cat_slope_MAX',
              'awc_huc12s_cat_slope_RANGE','awc_huc12s_cat_slope_MEAN', 'awc_huc12s_cat_slope_STD', 'awc_huc12s_cat_slope_SUM', 'awc_huc12s_cat_slope_MEDIAN', 'awc_huc12s_cat_slope_PCT90',
              'awc_huc12s_cat_elev_COUNT', 'awc_huc12s_cat_elev_AREA', 'awc_huc12s_cat_elev_MIN', 'awc_huc12s_cat_elev_MAX', 'awc_huc12s_cat_elev_RANGE', 'awc_huc12s_cat_elev_MEAN', 'awc_huc12s_cat_elev_STD',
              'awc_huc12s_cat_elev_SUM', 'awc_huc12s_cat_elev_VARIETY', 'awc_huc12s_cat_elev_MAJORITY', 'awc_huc12s_cat_elev_MINORITY', 'awc_huc12s_cat_elev_MEDIAN', 'awc_huc12s_cat_elev_PCT90',
              'awc_huc12s_wtd_elev_COUNT', 'awc_huc12s_wtd_elev_AREA', 'awc_huc12s_wtd_elev_MIN', 'awc_huc12s_wtd_elev_MAX', 'awc_huc12s_wtd_elev_RANGE', 'awc_huc12s_wtd_elev_MEAN',
              'awc_huc12s_wtd_elev_STD', 'awc_huc12s_wtd_elev_SUM', 'awc_huc12s_wtd_elev_VARIETY', 'awc_huc12s_wtd_elev_MAJORITY', 'awc_huc12s_wtd_elev_MINORITY',
              'awc_huc12s_wtd_elev_MEDIAN', 'awc_huc12s_wtd_elev_PCT90', 'awc_huc12s_wtd_slope_COUNT', 'awc_huc12s_wtd_slope_AREA', 'awc_huc12s_wtd_slope_MIN', 'awc_huc12s_wtd_slope_MAX',
              'awc_huc12s_wtd_slope_RANGE', 'awc_huc12s_wtd_slope_MEAN', 'awc_huc12s_wtd_slope_STD', 'awc_huc12s_wtd_slope_SUM', 'awc_huc12s_wtd_slope_MEDIAN', 'awc_huc12s_wtd_slope_PCT90',
              'non_north_area', 'north_area', 'awc_huc12s_wtd_north_per', 'non_wetland_area', 'wetland_area', 'awc_huc12s_wtd_wet_per',
              'awc_huc12s_wtd_lake_area_sqm', 'awc_huc12s_wtd_lake_per', 'awc_huc12s_wtd_glacier_area_sqm', 'awc_huc12s_wtd_glacier_per' ]
#Create list of duplicate column names and drop
drop_cols = ['cat_ID_txt_y', 'region_y', 'cat_ID_txt_x_2', 'region_x_2', 'region_y_2', 'cat_ID_txt_y_2', 'region_x_3',
             'cat_ID_txt_x_3', 'cat_ID_txt_y_3', 'FType', 'region_y_3','cat_ID_txt_x_4', 'O1Region', 'region_x_4',
             'cat_ID_y', 'cat_ID_txt_y_4', 'region_y_4']
df_final.drop(columns=drop_cols, axis = 1, inplace=True)
#rename columns
df_final.rename({'cat_ID_txt_x':'cat_ID_txt','cat_ID_x':'cat_ID','region_x':'region'},axis=1, inplace=True)
#Recalculate cat_ID
df_final['cat_ID'] = df_final['cat_ID_txt'].astype(np.float64)
# reorder cols
df_final = df_final.reindex(columns=final_cols)
df_final

In [ ]:
# Export merged dataframe to csv
cov_csv_out = os.path.join(outdir,'AKSSF_AWC_HUC12s_Covariates.csv')
df_final.to_csv(cov_csv_out, encoding = 'utf-8')
print('Export all covariates dataframe to csv complete')


In [25]:
bbay_df = df_final.filter(like='Bristol_Bay', axis = 0)
bbay_df

,cat_ID_txt,cat_ID,region,awc_huc12s_cat_slope_COUNT,awc_huc12s_cat_slope_AREA,awc_huc12s_cat_slope_MIN,awc_huc12s_cat_slope_MAX,awc_huc12s_cat_slope_RANGE,awc_huc12s_cat_slope_MEAN,awc_huc12s_cat_slope_STD,...,non_north_area,north_area,awc_huc12s_wtd_north_per,non_wetland_area,wetland_area,awc_huc12s_wtd_wet_per,awc_huc12s_wtd_lake_area_sqm,awc_huc12s_wtd_lake_per,awc_huc12s_wtd_glacier_area_sqm,awc_huc12s_wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,


In [26]:
kod_df = df_final.filter(like='Kodiak', axis = 0)
kod_df

,cat_ID_txt,cat_ID,region,awc_huc12s_cat_slope_COUNT,awc_huc12s_cat_slope_AREA,awc_huc12s_cat_slope_MIN,awc_huc12s_cat_slope_MAX,awc_huc12s_cat_slope_RANGE,awc_huc12s_cat_slope_MEAN,awc_huc12s_cat_slope_STD,...,non_north_area,north_area,awc_huc12s_wtd_north_per,non_wetland_area,wetland_area,awc_huc12s_wtd_wet_per,awc_huc12s_wtd_lake_area_sqm,awc_huc12s_wtd_lake_per,awc_huc12s_wtd_glacier_area_sqm,awc_huc12s_wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,


In [27]:
pws_df = df_final.filter(like='Prince', axis = 0)
pws_df

,cat_ID_txt,cat_ID,region,awc_huc12s_cat_slope_COUNT,awc_huc12s_cat_slope_AREA,awc_huc12s_cat_slope_MIN,awc_huc12s_cat_slope_MAX,awc_huc12s_cat_slope_RANGE,awc_huc12s_cat_slope_MEAN,awc_huc12s_cat_slope_STD,...,non_north_area,north_area,awc_huc12s_wtd_north_per,non_wetland_area,wetland_area,awc_huc12s_wtd_wet_per,awc_huc12s_wtd_lake_area_sqm,awc_huc12s_wtd_lake_per,awc_huc12s_wtd_glacier_area_sqm,awc_huc12s_wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,


In [28]:
ci_df = df_final.filter(like='Cook', axis = 0)
ci_df

,cat_ID_txt,cat_ID,region,awc_huc12s_cat_slope_COUNT,awc_huc12s_cat_slope_AREA,awc_huc12s_cat_slope_MIN,awc_huc12s_cat_slope_MAX,awc_huc12s_cat_slope_RANGE,awc_huc12s_cat_slope_MEAN,awc_huc12s_cat_slope_STD,...,non_north_area,north_area,awc_huc12s_wtd_north_per,non_wetland_area,wetland_area,awc_huc12s_wtd_wet_per,awc_huc12s_wtd_lake_area_sqm,awc_huc12s_wtd_lake_per,awc_huc12s_wtd_glacier_area_sqm,awc_huc12s_wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004200000901,75004200000901,75004200000901.00,Cook_Inlet,3516.00,351600.00,0.00,46.44,46.44,12.45,9.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200001724,75004200001724,75004200001724.00,Cook_Inlet,1410.00,141000.00,0.00,20.08,20.08,4.58,2.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200001726,75004200001726,75004200001726.00,Cook_Inlet,196.00,19600.00,0.00,3.44,3.44,0.82,1.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200001493,75004200001493,75004200001493.00,Cook_Inlet,525.00,52500.00,0.00,30.78,30.78,10.86,6.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200004105,75004200004105,75004200004105.00,Cook_Inlet,458.00,45800.00,0.00,16.35,16.35,4.50,3.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75005400000004,75005400000004,75005400000004.00,Cook_Inlet,34837.00,3483700.00,0.00,6.03,6.03,0.66,1.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75005400001431,75005400001431,75005400001431.00,Cook_Inlet,4333.00,433300.00,0.00,39.12,39.12,7.94,4.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75005400031008,75005400031008,75005400031008.00,Cook_Inlet,17011.00,1701100.00,0.00,46.25,46.25,5.63,6.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
cop_df = df_final.filter(like='Copper', axis = 0)
cop_df

,cat_ID_txt,cat_ID,region,awc_huc12s_cat_slope_COUNT,awc_huc12s_cat_slope_AREA,awc_huc12s_cat_slope_MIN,awc_huc12s_cat_slope_MAX,awc_huc12s_cat_slope_RANGE,awc_huc12s_cat_slope_MEAN,awc_huc12s_cat_slope_STD,...,non_north_area,north_area,awc_huc12s_wtd_north_per,non_wetland_area,wetland_area,awc_huc12s_wtd_wet_per,awc_huc12s_wtd_lake_area_sqm,awc_huc12s_wtd_lake_per,awc_huc12s_wtd_glacier_area_sqm,awc_huc12s_wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
